In [1]:
# This demo implements the recursive linearization method.
# The frequency marching step is 1/frequency.

from IPython.display import clear_output
from data_generator import DataGenerator
from medium_generator import MediumGenerator, disk_func, cosine_func
import numpy as np
from numpy.linalg import lstsq
from ngsolve import * 
from ngsolve.webgui import Draw

dg = DataGenerator(maxh = (0.1, 0.3))

medium = MediumGenerator(cosine_func) # MediumGenerator(disk_func)

background_params = [{"type": 0, "x": 0.0, "y": 0.0, "r": 0.5, "v": 0.0}]
background_permittivity = medium.generate(background_params)



params  = [{"type": 0, "x": 0.6 ,"y": 0.4, "r": 0.3, "v": 1}, 
           {"type": 0, "x": 0.7, "y": -0.6, "r": 0.3, "v": 2},
           {"type": 0, "x": -0.8, "y": -0.3, "r": 0.3, "v": 3},
           {"type": 0, "x": -0.5, "y": 0.5, "r": 0.3, "v": 4}]

permittivity  = medium.generate(params)


Mesh generation took 0.17480671405792236 seconds


In [2]:
# direction_angles = np.linspace(0, 2 * np.pi, 32, endpoint=False)

# cauchy_data = dg.generate_cauchy_data(2 * pi, direction_angles, permittivity)

In [3]:
n_dir = 16

direction_angles = np.linspace(0, 2 * np.pi, n_dir, endpoint=False)

A = Matrix(len(direction_angles) ** 2, dg.fes.ndof, complex=True)
b = Vector(len(direction_angles) ** 2, complex=True)

freq = 0.0

for iter in range(40):
    print(iter)

    l = 0
    freq += 0.1

    print(freq)
    print('error squared:', Integrate((background_permittivity-permittivity)*Conj(background_permittivity-permittivity), dg.mesh).real)


    direction_angles = np.linspace(0, 2 * np.pi, n_dir, endpoint=False)

    # direction_angles += np.random.random()*2*np.pi

    for i in range(n_dir):

        kx = 2 * pi * freq * cos(direction_angles[i])
        ky = 2 * pi * freq * sin(direction_angles[i])

        psi = CF((exp(1j * kx * x) * exp(1j * ky * y)))

        u_scat = dg.solve(kx, ky, permittivity)
        uB_scat_psi = dg.solve(kx, ky, background_permittivity)


        for j in range(n_dir):
            p_kx = 2 * pi * freq * cos(direction_angles[j])
            p_ky = 2 * pi * freq * sin(direction_angles[j])

            phi = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))

            uB_scat_phi = dg.solve(p_kx, p_ky, background_permittivity)

            true_val = Integrate( (permittivity - background_permittivity) * (uB_scat_phi + phi) *  (u_scat + psi), dg.mesh)

            test_func = dg.fes.TestFunction()

            linear_form = LinearForm(dg.fes)

            linear_form += test_func * (uB_scat_phi + phi) *  (uB_scat_psi + psi) * (IfPos((x)**2 + (y)**2 - (1.5) **2,
                    0,
                    1))  * dx

            linear_form.Assemble()

            A.NumPy()[l, :] = linear_form.vec.FV().NumPy()
            b[l] = true_val
            l += 1
    
    v = lstsq(A.NumPy(),  b.NumPy(), rcond=1e-2)[0]
    
    permittivity_update =  GridFunction(dg.fes)
    permittivity_update.vec.data = v
    background_permittivity = background_permittivity + permittivity_update
    

    scene = Draw(background_permittivity, dg.mesh)
    clear_output() # redraw the scene with a new height
    scene.Draw(height="50vh")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

In [4]:
# error plot
scene = Draw(background_permittivity-permittivity, dg.mesh)
clear_output() # redraw the scene with a new height
scene.Draw(height="50vh")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Complex': {'phase': 0.0, 'sp…

In [5]:
# L2 error square
Integrate((background_permittivity-permittivity)*Conj(background_permittivity-permittivity), dg.mesh)

(0.0056939592774422855-6.767169750867377e-22j)